In [1]:
import boto3
import pandas as pd
from sqlalchemy import create_engine
from io import StringIO

# Function to load data from S3
def load_data_from_s3(bucket_name, file_key):
    print("Connecting to S3 to retrieve data...")
    s3_client = boto3.client('s3')
    csv_obj = s3_client.get_object(Bucket=bucket_name, Key=file_key)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_string), low_memory=False)  # Adjust dtype or parsing options as needed
    print("Data successfully loaded from S3.")
    return df

# Function to insert data into RDS using SQLAlchemy for efficient batch inserts
def insert_data_to_rds(df, database_details):
    print("Connecting to RDS database...")
    # Database connection URL
    database_url = f"mysql+pymysql://{database_details['user']}:{database_details['password']}@{database_details['host']}/{database_details['database']}"
    engine = create_engine(database_url)

    print("Inserting data into the database...")
    df.to_sql('listings', con=engine, index=False, if_exists='append', chunksize=500)  # Adjust chunksize based on your needs
    print("Data successfully inserted into the database.")

# Main execution
bucket_name = 'group-1-final-project-finalsdata'
file_key = 'final_airbnb_dataset.csv'
database_details = {
    'host': 'database-2.c5s0q6g8qg6i.us-east-2.rds.amazonaws.com',
    'user': 'admin',
    'password': 'anveshalluri',
    'database': 'airbnb_db'
}

print("Script started.")
data = load_data_from_s3(bucket_name, file_key)
insert_data_to_rds(data, database_details)
print("Script executed successfully.")


Script started.
Connecting to S3 to retrieve data...
Data successfully loaded from S3.
Connecting to RDS database...
Inserting data into the database...
Data successfully inserted into the database.
Script executed successfully.
